# XGB算法梳理
## 0.CART树

0.1 CART是决策树算法之一，它可用于分类问题也可用于回归问题。它与ID3、C4.5等算法不同的是，在节点分支的时候，它每次仅分成左、右两个节点，即它假设决策树是二叉树。另外，它在选择最优分割变量与最优分割点的准则也有所不同。

0.2 CART树的生成就是递归地构建二叉决策树的过程。对回归树用平方误差最小化准则，对分类树用基尼指数最小化准则，进行特征选择，生成二叉树。

![cart](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/xgboost-cart.jpg)

在这里，平方误差就是叶子节点的输出值与样本实际值偏差的平方和，概念比较清晰不过多赘述。至于基尼指数，通俗来讲就是，在一个分支里，随机抽取两个样本，这两个样本所属类别不一样的概率，如果基尼指数越小，那么这个分支的纯度就越高。具体看看基尼指数怎么定义的：       分类问题中，假设有K个类，样本点属于k类的概率为  ，则概率分布的基尼指数定义为：

$Gini(p)=\sum_{k=1}^{K}{p_{k}(1-p_{k})=1-\sum_{k=1}^{K}{p_{k}^{2}}}$

对于给定的样本集合D，其基尼指数为：

$Gini(D)=1-\sum_{k=1}^{K}{(\frac{ |c_{k}|}{|D|})^{2}}$

这里， $C_k$ 是D中属于第k类的样本子集，K是类的个数。

如果样本集合D根据特征A是否取某一可能值a被分割成 $D_1$ 和 $D_2$ 两部分，即

$D_1=\{(x,y)\in D|A(x)=a\},D_{2}=D-D_{1}$

则在特征A条件下，集合D的基尼指数定义为：

$Gain(D,A)=\frac{|D_{1}|}{|D|}Gini(D_{1})+\frac{|D_{2}|}{|D|}Gini(D_{2})$

基尼指数Gini(D)表示集合的不确定性，基尼指数Gini(D,A)表示经A=a分割后集合D的不确定性。基尼指数值越大，样本集合的不确定性也就越大。
## 1.算法原理
1.1.算法思想

- XGBoost算法的基本思想与GBDT类似，不断地地进行特征分裂来生长一棵树，每一轮学习一棵树，其实就是去拟合上一轮模型的预测值与实际值之间的残差。当我们训练完成得到k棵树时，我们要预测一个样本的分数，其实就是根据这个样本的特征，在每棵树中会落到对应的一个叶子节点，每个叶子节点就对应一个分数，最后只需将每棵树对应的分数加起来就是该样本的预测值。

1.2.模型的函数形式

给定数据集D={ $x_i,y_i$}，XGBoost进行additive training，学习K棵树，采用以下函数对样本进行预测：

$\tilde{y}=\phi(x_{i})=\sum_{k=1}^{K}{f_{k}(x_{i})}，f_{k}\in\Gamma$

这里 $\Gamma$ 是假设空间，f(x)是回归树：
$\Gamma=\{ f(x)=w_{q(x)}\}(q:R^m \rightarrow T,w \in R^T)$

q(x)表示样本x分到了某个叶子节点上，w是叶子节点的分数，所以$w_{q(x)}$ 表示回归树对样本的预测值。

例子：预测一个人是否喜欢玩泥沙:

![cart](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/gbdt_tree.jpg)

上面训练出了2棵树，小男孩的预测分数就是两棵树中小男孩所落到的节点的分数相加，老头子的分数同理。

回归树预测输出的是实数分数，可用于回归、分类、排序等任务。对于回归问题，预测分数可以直接作为目标值，对于分类问题，需要映射成概率，比如采用逻辑函数 $\sigma(z)=-\frac{1}{1+e^{-z}}$
## 2.损失函数

2.1 每一轮训练一棵树，都是为了使得损失函数能够极小化，也就是每一棵树训练要达到的目标。XGBoost的损失函数（目标函数）与GBDT不同，它不仅仅是衡量了模型的拟合误差，还增加了正则化项，即对每棵树树复杂度的惩罚项，来限制树的复杂度，防止过拟合。

**参数空间中的目标函数：**

![cart](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/xgboost_target_func.jpg)

在函数空间中目标函数形式为： $L(\phi)=\displaystyle \sum_i l(\tilde y_i,y_i)+ \displaystyle \sum_k \Omega(f_k)$

其中，正则化项 $\Omega(f)=\gamma T+\frac{1}{2}\lambda||w||^2$ ，T为叶子节点个数，w为叶子节点分数。

误差函数的二阶泰勒展开：

- 第t次迭代后，模型的预测等于前t-1次模型预测加第t棵树的预测：$\hat y_i^{(t)}=\hat y^(t-1)+f_t(x_i)$

- 此时目标函数可写作：$L^{(t)}=\displaystyle \sum^n_{i=1} l(y_i,\hat y_i^{t-1}+f_t(x_i))+\Omega(f_t)$
   
公式中， $y_i,\tilde y_i^{t-1}$ 都已知，模型要学习的只有第t棵树 f_t

将误差函数在 $y_i^{t-1}$ 处进行二阶泰勒展开：$L^{(t)} \approx \displaystyle \sum_{i=1}^n [l(y_i,\hat y^{(t-1)})+g_i f_t(x_i)+\frac{1}{2}h_i f_t^2(x_i)]+\Omega(f_t)$
- 公式中

$g_i=\partial_{\hat y^{(t-1)}}l(y_i,\hat y^{(t-1)}) $ $h_i-\partial^2_{\hat y_{(t-1)}}l(y_i,\hat y^{(t-1)})$
- 把公式中的常数项去掉，得到：$\tilde L^{(t)}=\displaystyle \sum_{i=1}^n[g_if_t(x_i)_\frac{1}{2}h_if_t^2(x_i)]+\Omega(f_t)$
- 把 $f_t,\Omega(f_t)$ 写成树结构的形式，即把下式带入目标函数中$f(x)=w_{q(x)} \quad \Omega(f)=\gamma T+\frac{1}{2}\lambda||w||^2$
- 得到：$\tilde L^{(t)}=\displaystyle \sum_{i=1}^n[g_if_t(x_i)_\frac{1}{2}h_if_t^2(x_i)]+\Omega(f_t)$

=$\displaystyle\sum_{i=1}^n[g_iw_{q(x_i)}+\frac{1}{2}h_iw^2_q(x_i)]+\gamma T +\lambda\frac{1}{2}\displaystyle\sum_{j=1}^Tw_j^2$
![formula](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/xgboost_formula.jpg)
- 怎么统一起来呢？
定义每个叶节点j上的样本集合为 $I_j=\{i|q(x_i)=j\}$ ，则目标函数可以写成按叶节点累加的形式：$\tilde L^{(t)}=\displaystyle\sum_{j=1}^T[(\displaystyle\sum_{i\in I_j}g_i)w_j+\frac{1}{2}(\displaystyle\sum_{i\in I_j}h_i+\lambda)w_j^2]+\gamma T\\=\displaystyle\sum_{j=1}^T[G_jw_j+\frac{1}{2}(H_j+\lambda)w_j^2]+\gamma T$
- 如果确定了树的结构（即q(x)确定），为了使目标函数最小，可以令其导数为0，解得每个叶节点的最有预测分数为
$w_j^*=-\frac{G_j}{H_j+\lambda}$
- 代入目标函数，得到最小损失为：$\tilde L^*=-\frac{1}{2}\displaystyle\sum_{j=1}^T\frac{G_j^2}{H_j+\lambda}+\gamma T$

## 3.分裂结点算法

3.1 当回归树的结构确定时，我们已经推导出其最优的节点分数及对应的最小损失值，问题是怎么确定树的结构呢？
- XGBoost的打分函数：
![formula](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/xgboost_loss_func.jpg)
标红部分衡量了每个叶子节点对总体损失的贡献，我们希望损失函数越小越好，则标红部分的值越大越好。

- 因此，对一个叶子节点进行分裂，分裂前后的增益定义为： $Gain=\frac{G_{L}^{2}}{H_{L}+\lambda}+ \frac{G_{R}^{2}}{H_{R}+\lambda}-\frac{(G_{L}+G_{R})^{2}}{H_{L}+H_{R}+\lambda}-\gamma$
![formula](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/xgboost-formula-explain.png)



Gain的值越大，分裂后L减少越多。所当对一个叶节点分割时，计算所有候选特征对应的Gain选取Gain最大的进行分割。
- 1.精确算法：遍历所有特征的所有可能的分割点，计算gain值，选取值最大的特征去分割：
![formula](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/xgboost_algorithm_1.jpg)
- 2.近似算法，对每个特征，只考察分位点，减少计算复杂度
![formula](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/xgboost_algorithm_2.jpg)
Global：学习每棵树前，提出候选切分点

Local：每次分列前，重新提出候选切分点
- 实际上，XGBoost不是简单地按照样本个数进行分位，而是以二阶导数值作为权重。
## 4.正则化

XGBoost有几种防止过拟合的正则化方法。

4.1.在目标函数中，不仅包含了模型的拟合误差函数，还增加了关于每棵树复杂度的惩罚项，即叶子节点的个数以及叶子节点分数的平方项，限制了树的复杂度。

4.2.像GBDT那样，可以对每个模型乘上一个步长a，a∈(0,1]，用来降低每个模型对预测的贡献。

4.3.可以行采样与列采样，与随机森林类似。

## 5.对缺失值处理

XGBoost在选择最优分裂点的时候，不考虑该特征的缺失样本。但在后面对样本划分中，会分别将该特征的缺失样本放到左、右节点中，分别计算Gain值，哪边大就把该样本划分到哪一边。如果在训练集中，该特征没有出现缺失样本，但在预测的时候出现缺失样本了，则默认将该样本划分到右节点中。具体方法：

![xgboost_fill_loss_feature](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/xgboost_fill_loss_feature.jpg)
## 6.优缺点

6.1.XGBoost与GBDT相比，其优势：将树模型的复杂度加入到正则项中，来避免过拟合，因此泛化性能会优于GBDT。损失函数用泰勒展开式展开，同时用到了一阶和二阶导数，可以加快优化速度。GBDT只支持CART作为基学习器，XGBoost还支持线性分类器作为基学习器。引进了特征子采样，像随机森林那样，既能避免过拟合，又能减少计算。在寻找最优分割点时，考虑到传统的贪心算法效率较低，实现了一种近似贪心算法，用来加速和减少内存小号，除此之外，还考虑了稀疏数据集合缺失值的处理。XGBoost支持并行处理。XGBoost的并行不是模型生成的并行，而是在特征上的并行，将特征排序后以block的形式存储在内存中，在后面迭代重复使用这个结构。这个block也使得并行化成为了可能，其次在节点分裂时，计算每个特征的增益，最终选择增益最大的那个特征去做分割，那么各个特征的增益计算就可以开多线程进行。

6.2.与lightGBM相比的不足点：XGBoosting采用预排序，在迭代之前，对结点的特征做预排序，遍历选择最优分割点，数据量大时，贪心法耗时，LightGBM方法采用histogram算法，占用的内存低，数据分割的复杂度更低。XGBoosting采用level-wise生成决策树，同时分裂同一层的叶子，从而进行多线程优化，不容易过拟合，但很多叶子节点的分裂增益较低，没必要进行跟进一步的分裂，这就带来了不必要的开销；LightGBM采用深度优化，leaf-wise生长策略，每次从当前叶子中选择增益最大的结点进行分裂，循环迭代，但会生长出更深的决策树，产生过拟合，因此引入了一个阈值进行限制，防止过拟合。
## 7.应用场景
## 8.sklearn参数

xgboost 有很多可调参数，具有极大的自定义灵活性。比如说：

（1）objective [ default=reg:linear ] 定义学习任务及相应的学习目标，可选的目标函数如下：

“reg:linear” –线性回归。

“reg:logistic” –逻辑回归。

“binary:logistic” –二分类的逻辑回归问题，输出为概率。

“multi:softmax” –处理多分类问题，同时需要设置参数num_class（类别个数）

（2）’eval_metric’ The choices are listed below，评估指标:

“rmse”: root mean square error

“logloss”: negative log-likelihood

(3)max_depth [default=6] 数的最大深度。缺省值为6 ，取值范围为：[1,∞]



参考：https://www.zhihu.com/search?type=content&q=XGB算法梳理